### 확률 → 정보량 → 엔트로피 → 크로스엔트로피 → KL 발산

- 확률
- 정보량: 한 사건의 놀라움 -log(p) --> 확률의 역수
- 엔트로피: (진짜 분포가 가진) 평균 정보량 H(p_true)
- 크로스엔트로피: (진짜 분포를) 예측 분포로 코딩할 때 평균 비용 CE(p_true, p_pred)
- KL 발산: 예측이 진짜에서 얼마나 벗어났는지 KL = CE - H(true)
- 손실함수(BCE): 실제 라벨에 대한 크로스엔트로피(= Negative Log-Likelihood) 를 평균낸 것


In [2]:
import numpy as np

# -----------------------
# (A) 확률 -> 정보량
# -----------------------
p = 0.8  # 어떤 사건(예: 연체=1)이 일어날 확률
info = -np.log(p)       # 정보량(자연로그, 단위: nat)
info_bits = -np.log2(p) # 정보량(로그2, 단위: bit)

print("확률 p =", p)
print("정보량 -log(p) =", info, "(nat)")
print("정보량 -log2(p) =", info_bits, "(bit)")

# -----------------------
# (B) 엔트로피 (베르누이 분포의 불확실성)
# H(p) = -[ p log p + (1-p) log (1-p) ]
# -----------------------
def bern_entropy(p):   # 확률 𝑝를 입력으로 받아 베르누이 확률변수의 엔트로피 𝐻(𝑝)를 반환하는 함수
    p = np.clip(p, 1e-12, 1-1e-12)  # 수치 안정성 확보 (np.clip),  NaN / inf 에러가 발생
    return -(p*np.log(p) + (1-p)*np.log(1-p))

print("\n엔트로피 H(p) =", bern_entropy(p))  # 주어진 p에 대해 엔트로피 계산 . 결과 출력

# -----------------------
# (C) 크로스엔트로피 / KL 발산
# true 분포 p_true, 예측 분포 p_pred(모델 확률)
# CE(p_true, p_pred) = -[ p_true log p_pred + (1-p_true) log (1-p_pred) ]
# KL(p_true || p_pred) = CE - H(p_true)
# -----------------------
p_true = 0.30   # 실제 연체 비율(진짜 분포)
p_pred = 0.60   # 모델이 믿는 연체 확률(예측 분포)

def bern_cross_entropy(p_true, p_pred):
    p_true = np.clip(p_true, 1e-12, 1-1e-12)
    p_pred = np.clip(p_pred, 1e-12, 1-1e-12)
    return -(p_true*np.log(p_pred) + (1-p_true)*np.log(1-p_pred))

H_true = bern_entropy(p_true)
CE = bern_cross_entropy(p_true, p_pred)
KL = CE - H_true

print("\n[베르누이 분포 기준]")
print("H(true) =", H_true)
print("CE(true, pred) =", CE)
print("KL(true || pred) =", KL)

# -----------------------
# (D) 손실함수와 연결: Binary Cross Entropy (샘플 단위)
# y in {0,1}, p_hat = P(y=1|x)
# loss = -[ y log p_hat + (1-y) log(1-p_hat) ]
# -----------------------
y = np.array([0, 1, 0, 1, 1])                       # 실제 라벨
p_hat = np.array([0.10, 0.80, 0.40, 0.60, 0.95])    # 모델이 내는 "연체 확률"

p_hat = np.clip(p_hat, 1e-12, 1-1e-12)
bce_per_sample = -(y*np.log(p_hat) + (1-y)*np.log(1-p_hat))
bce_mean = bce_per_sample.mean()

print("\n[BCE: 이진분류 손실]")
print("샘플별 BCE =", np.round(bce_per_sample, 4))
print("평균 BCE =", np.round(bce_mean, 4))


확률 p = 0.8
정보량 -log(p) = 0.2231435513142097 (nat)
정보량 -log2(p) = 0.3219280948873623 (bit)

엔트로피 H(p) = 0.5004024235381879

[베르누이 분포 기준]
H(true) = 0.6108643020548935
CE(true, pred) = 0.7946511994417057
KL(true || pred) = 0.1837868973868122

[BCE: 이진분류 손실]
샘플별 BCE = [0.1054 0.2231 0.5108 0.5108 0.0513]
평균 BCE = 0.2803


#### 다중분류(Softmax) 예시: 엔트로피/크로스엔트로피/KL/손실

- 정보이론 → 크로스엔트로피 → KL divergence → 딥러닝 다중분류 손실

In [3]:
import numpy as np

# -----------------------
# 다중분류 분포 (예: 클래스 3개)
# true 분포 q (원-핫 or 부드러운 분포)
# pred 분포 p (softmax 출력 확률)
# -----------------------

# 정답 분포 q 와 예측 분포 p
q = np.array([0.0, 1.0, 0.0])    
p = np.array([0.2, 0.7, 0.1])     # 모델 예측 확률 (softmax 출력)

eps = 1e-12  # 수치 안정성 처리 , log(0) → -inf
q = np.clip(q, eps, 1.0)
p = np.clip(p, eps, 1.0)

# 정보량(정답 클래스에 대한) = -log(p_true_class)
info_true_class = -np.log(p[1])   # q가 2번 클래스이므로 p[1]

# 엔트로피 H(q) = -sum q log q  (원-핫이면 거의 0)
H_q = -np.sum(q * np.log(q))

# 크로스엔트로피 CE(q, p) = -sum q log p  (원-핫이면 -log(p_true_class))
CE_qp = -np.sum(q * np.log(p))

# KL(q || p) = sum q log(q/p) = CE - H(q)
KL_qp = CE_qp - H_q

print("정보량(정답 클래스) =", info_true_class)
print("H(q) =", H_q)
print("CE(q,p) =", CE_qp)
print("KL(q||p) =", KL_qp)

# -----------------------
# 손실함수와 연결: Categorical Cross Entropy (다중분류 손실)
# 원-핫 정답 q 와 예측 확률 p의 크로스엔트로피가 곧 손실
# -----------------------
loss = CE_qp
print("\n[다중분류 손실 = Categorical Cross Entropy] =", loss)


정보량(정답 클래스) = 0.35667494393873245
H(q) = 5.526204223185709e-11
CE(q,p) = 0.3566749439426445
KL(q||p) = 0.35667494388738247

[다중분류 손실 = Categorical Cross Entropy] = 0.3566749439426445


### Softmax → 확률 p 만들기까지 “점수→확률” 연결

In [4]:
import numpy as np

def softmax(z):
    z = z - np.max(z)              # 안정화
    expz = np.exp(z)
    return expz / np.sum(expz)

scores = np.array([1.2, 2.0, 0.1])  # 모델의 점수(score, logit)
p = softmax(scores)

print("scores =", scores)
print("softmax 확률 p =", np.round(p, 4))


scores = [1.2 2.  0.1]
softmax 확률 p = [0.281  0.6254 0.0935]


- softmax 함수가 점수(score, logit)를 확률로 바꾸는 과정
- 점수를 지수(exp)로 바꾼 뒤 전체 합으로 나누어 확률 분포로 만든다.